### 1. Trying out sentiment analysis models

In [11]:
from transformers import TFRobertaForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax
import logging
from transformers import logging as transformers_logging

transformers_logging.set_verbosity_error()
logging.getLogger("transformers").setLevel(logging.ERROR)

labels = {
    0 : 'negative',
    1 : 'neutral',
    2 : 'positive'
}
def test_sentiment_score(text, tokenizer, model):
    encoded_input = tokenizer(text, return_tensors='tf')
    output = model(encoded_input)
    scores = output[0][0].numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    ranked_labels = [labels[i] for i in ranking]
    return (ranked_labels[0])

In [16]:
import re
 
text = "I hate children"
model_roberta_latest = "cardiffnlp/twitter-roberta-base-sentiment-latest"
model_roberta_older = "cardiffnlp/twitter-roberta-base-sentiment"
model_name_bertweet = "finiteautomata/bertweet-base-sentiment-analysis"


tokenizer_latest = AutoTokenizer.from_pretrained(model_roberta_latest)
model_latest = TFRobertaForSequenceClassification.from_pretrained(model_roberta_latest)

tokenizer_older = AutoTokenizer.from_pretrained(model_roberta_older)
model_older = TFRobertaForSequenceClassification.from_pretrained(model_roberta_older)

tokenizer_bertweet = AutoTokenizer.from_pretrained(model_name_bertweet)
model_bertweet= TFAutoModelForSequenceClassification.from_pretrained(model_name_bertweet)

sentiment_score_new = test_sentiment_score(text, tokenizer_latest, model_latest)
sentiment_score_old = test_sentiment_score(text, tokenizer_older, model_older)
sentiment_score_bertweet = test_sentiment_score(text, tokenizer_bertweet, model_bertweet)


print(f"Sentiment score ({re.search(r'[^/]+$', model_roberta_latest).group(0)}): {sentiment_score_new}")
print(f"Sentiment score ({re.search(r'[^/]+$', model_roberta_older).group(0)}): {sentiment_score_old}")
print(f"Sentiment score ({re.search(r'[^/]+$', model_name_bertweet).group(0)}): {sentiment_score_bertweet}")

Sentiment score (twitter-roberta-base-sentiment-latest): negative
Sentiment score (twitter-roberta-base-sentiment): negative
Sentiment score (bertweet-base-sentiment-analysis): negative


### 2. Setting up dataframe for accuracy calculation

In [1]:
import pandas as pd

df_test = pd.read_csv('twitter_validation.csv')
df_test.head()

,tweet_id,entity,sentiment,tweet
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...


In [34]:

df_test = df_test.drop(df_test[df_test['sentiment'] == 'Irrelevant'].index)
df_test = df_test.head(1000) #Only doing this on the first 1000 tweets, to save time

In [35]:
pd.set_option('display.max_colwidth', None)  
df_test['roberta_old'] = df_test['tweet'].apply(lambda x: test_sentiment_score(x, tokenizer_older, model_older))
df_test['roberta_new'] = df_test['tweet'].apply(lambda x: test_sentiment_score(x, tokenizer_latest, model_latest))
df_test['bertweet'] = df_test['tweet'].apply(lambda x: test_sentiment_score(x, tokenizer_bertweet, model_bertweet))
df_test

,tweet_id,entity,sentiment,tweet,roberta_old,roberta_new,bertweet
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects claims company acted like a 'drug dealer' bbc.co.uk/news/av/busine…,neutral,positive,negative
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it functions so poorly on my @SamsungUS Chromebook? 🙄,negative,positive,negative
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking, it's a truly awful game.",negative,positive,negative
4,4433,Google,Neutral,Now the President is slapping Americans in the face that he really did commit an unlawful act after his acquittal! From Discover on Google vanityfair.com/news/2020/02/t…,negative,positive,negative
5,6273,FIFA,Negative,"Hi @EAHelp I’ve had Madeleine McCann in my cellar for the past 13 years and the little sneaky thing just escaped whilst I was loading up some fifa points, she took my card and I’m having to use my paypal account but it isn’t working, can you help me resolve it please?",negative,positive,negative
...,...,...,...,...,...,...,...
993,314,Amazon,Negative,Please explain how this is possible! How can they let companies overcharge for this? It’s a scam! @amazon shame on you! pic.twitter.com/MQB7BUPthx,negative,positive,negative
994,9701,PlayStation5(PS5),Positive,"Good on Sony. As much as I want to see the new PS5, what's going on right now is so much more important.",positive,positive,positive
997,2652,Borderlands,Positive,Today sucked so it’s time to drink wine n play borderlands until the sun comes up so I can hate myself all day tomorrow.,negative,positive,negative
998,8069,Microsoft,Positive,Bought a fraction of Microsoft today. Small wins.,positive,positive,neutral


In [36]:
df_test.loc[df_test['sentiment'] == 'Neutral', 'sentiment'] = 'neutral'
df_test.loc[df_test['sentiment'] == 'Positive', 'sentiment'] = 'positive'
df_test.loc[df_test['sentiment'] == 'Negative', 'sentiment'] = 'negative'

In [37]:
df_test.head()

,tweet_id,entity,sentiment,tweet,roberta_old,roberta_new,bertweet
1,352,Amazon,neutral,BBC News - Amazon boss Jeff Bezos rejects claims company acted like a 'drug dealer' bbc.co.uk/news/av/busine…,neutral,positive,negative
2,8312,Microsoft,negative,@Microsoft Why do I pay for WORD when it functions so poorly on my @SamsungUS Chromebook? 🙄,negative,positive,negative
3,4371,CS-GO,negative,"CSGO matchmaking is so full of closet hacking, it's a truly awful game.",negative,positive,negative
4,4433,Google,neutral,Now the President is slapping Americans in the face that he really did commit an unlawful act after his acquittal! From Discover on Google vanityfair.com/news/2020/02/t…,negative,positive,negative
5,6273,FIFA,negative,"Hi @EAHelp I’ve had Madeleine McCann in my cellar for the past 13 years and the little sneaky thing just escaped whilst I was loading up some fifa points, she took my card and I’m having to use my paypal account but it isn’t working, can you help me resolve it please?",negative,positive,negative


### 3. Checking the accuracy by comparing model labels with human labels (on 100 first tweets)

In [38]:
matches = df_test['sentiment'] == df_test['roberta_old']
percentage_matches = matches.mean() * 100
print(f"Accuracy of roberta_old model: {percentage_matches:.2f}%")

Accuracy of roberta_old model: 65.58%


In [39]:
matches = df_test['sentiment'] == df_test['roberta_new']
percentage_matches = matches.mean() * 100
print(f"Accuracy of roberta_new model: {percentage_matches:.2f}%")

Accuracy of roberta_new model: 29.95%


In [40]:
matches = df_test['sentiment'] == df_test['bertweet']
percentage_matches = matches.mean() * 100
print(f"Accuracy of bertweet model: {percentage_matches:.2f}%")

Accuracy of bertweet model: 63.65%
